In [1]:
# Standard 
import os, sys , copy ,shutil, logging, csv, json 

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Import all the streamm modules
from streamm import *

%load_ext autoreload
%autoreload 2

In [2]:
os.chdir('/Users/tkemper/Projects/DApolymers/scratch/D51_00R300R3_A23_R4__n1wB_R41n1wBR41n1wBR41n1wB__n5_x173_prod1_min2_grp1_remin1_res_et2')

In [3]:
mol1 = buildingblock.Container('mol1')

In [4]:
mol1.read_cply()

In [5]:
group_i_id = 'mol'

In [6]:
mol1.group_prop(group_i_id,group_i_id)

In [7]:
        groupset_mol = mol1.groupsets[group_i_id]
        groupset_mol.calc_cent_mass()
        groupset_mol.calc_radius()
        groupset_mol.group_pbcs()

In [8]:
os.chdir('/Users/tkemper/Projects/DApolymers/scratch/htermtest')

In [9]:
mol1.write_xyz('%s_pbcs.xyz'%(group_i_id))

In [10]:
def find_DA(prod1_strucC):
    

    prod1_strucC.propcompile_particles()
    part_df =  pd.DataFrame(prod1_strucC.prop_particles)
    ress = []
    
    for res_i in  part_df['resname']:
        if( res_i not in ress ):
            ress.append(res_i)
        
            print res_i

    res_D = None
    res_A = None
    for res_i in ress:
        if( res_i[:1] == 'D' ):
            res_D = res_i
        if( res_i[:1] == 'A' ):
            res_A = res_i
    if( res_D == None or res_A == None ):
        print " Search failed"
         
    
    sel_D = part_df['resname'] == res_D
    sel_A = part_df['resname'] == res_A
    sel_R2 = part_df['resname'] == 'R2'
    sel_R3 = part_df['resname'] == 'R3'
    sel_R4 = part_df['resname'] == 'R4'
    sel_THIO = part_df['resname'] == 'THIO'



    syC = part_df['symbol'] == 'C'
    syS = part_df['symbol'] == 'S'
    syN = part_df['symbol'] == 'N'
    syO = part_df['symbol'] == 'O'    
    syH = part_df['symbol'] == 'H'    
    syF = part_df['symbol'] == 'F'    
    ffCBang = part_df['fftype'] == 'C!'

    sub_DA = part_df[ (syC | syS  |syN | syO | syH | syF ) &  (sel_D | sel_A | sel_R2  | sel_R4 | ( sel_THIO & ffCBang) ) ]
    prod1_strucC.write_xyz_list(sub_DA.index,'sub_DA.xyz') 
    
    return sub_DA
    
    

In [11]:
sub_DA = find_DA(mol1)

D1
THIO
R3
A23
R4
R41


In [12]:
group_id = 'residue'

In [13]:
mol1.group_prop(group_id,group_id,particles_select=sub_DA.index)

In [14]:
groupset_i = mol1.groupsets[group_id]

In [15]:
groupset_i.calc_cent_mass()

In [16]:
groupset_i.calc_radius()

In [17]:
    et_cut = 0.0 
    groupset_i.properties['radius'] = []
    for gkey,group_i in groupset_i.groups.iteritems():
        if(  et_cut  > 0 ):
            group_i.properties['radius'] = et_cut 
        groupset_i.properties['radius'].append(group_i.properties['radius'])


In [18]:
group_i = groupset_i.groups[0]

In [20]:
        hb_length = 1.09
        hb_angle =  109.5
        tetrahedral_angle = np.deg2rad(hb_angle/2.0 )
        scale_vcross = np.sin(tetrahedral_angle) * hb_length
        scale_vadd = np.cos(tetrahedral_angle) * hb_length


In [21]:
self = group_i

In [120]:
        latticevec = self.strucC.lat._matrix


        hb_length = 1.09
        hb_angle =  109.5
        tetrahedral_angle = np.deg2rad(hb_angle/2.0 )


        scale_vcross = np.sin(tetrahedral_angle) * hb_length
        scale_vadd = np.cos(tetrahedral_angle) * hb_length
        
        
        HCC_angle = np.deg2rad(hb_angle )
        HCC_sin = np.sin(HCC_angle) #* hb_length
        HCC_cos = np.cos(HCC_angle) #* hb_length


        pt_H = structure.Atom('H')
        pt_H.properties = periodictable.element_number(1)
        pt_H.properties["fftype"] = "HC"
        pt_H.properties["resname"] = "TERM"
        pt_H.properties["mol"] = self.properties["mol"] 
        pt_H.properties["residue"] = self.properties["residue"] 

        original_ref_mod = []
        sub_ref_mod = []
        pid_i_mod = 0

        Htermed = self.strucC.getSubStructure(self.pkeys,tag='%s_hterm'%(self.tag))



In [121]:
        #def hterm_group(self,debug=False):
        """
        Hydrogen terminate group  

        (j)    (l)
            \  /
            (i)    - 109.5 deg
            /  \
          (k)   (m)
          
        Hydrogens will be added to sp3 carbons in a tetrahedral configuration

            ^         (l)
            |        /  |
       A    |   C  /    | 
            |    /      |
            |  /        |
            (i) -------->  
                B

        If atom i is bonded to two other atoms j and k
        r_ij = position_i - position_j
        r_ik = position_i - position_k
            A = r_ij x r_ik
            B = r_ij + r_ik

        For the angle (l)(i)(m) to be 109.5

        |A| = sin(  109.5/2 ) |C|
        |B| = cos(  109.5/2 ) |C|

        By scaling A and B they can be added to get the bonds between i and l and m

        r_il = A + B
        r_im = -A + B

        """
        
        for pkey_i in Htermed.particles.keys():
            NNAB_i = Htermed.bonded_nblist.calc_nnab(pkey_i)  # Group neighbors
            pkey_o = self.pkeys[pkey_i]
            NNAB_o = self.strucC.bonded_nblist.calc_nnab(pkey_o)# Structure container neighbors 
            if( NNAB_o != NNAB_i):
                # If there has been neighbors removed terminate 
                dB = NNAB_o - NNAB_i
                particle_i = Htermed.particles[pkey_i]
                r_i = Htermed.positions[pkey_i]
                r_ij_array = []
                for pkey_j in Htermed.bonded_nblist.getnbs(pkey_i):
                    r_j =  Htermed.positions[pkey_j]
                    r_ij = Htermed.lat.deltasq_pos(r_i,r_j)
                    r_ij_array.append(r_ij)

                    # print r_i,r_j,r_ij
                
                if( len(r_ij_array) != NNAB_i ):
                    error_line = " len(r_ij_array) {} != NNAB_i {} in groups.hterm() ".format(len(r_ij_array),NNAB_i)
                    sys.exit(error_line)
                    
                print ">hterm_group",NNAB_o,dB 
                
                if( NNAB_o == 3 and dB == 2 ):
                    '''
                    Choose vector cros_ik normal to plane of i,j and one nieghbor of j 
                    
                    
                    
                         cros_ik    H0
                          |        / 
                          |       /
                          j ---- i 
                        / 
                       / 
                    jk
                    
                    Scale vectore cros_ik by sin(109.5)
                    

                             H1
                            /  
                           /    
                    j----i  ----  dr_CC_s                        
                    
                                        
                    
                    add to vector dr_CC between i-j scaled by cos(109.5) 
                    to get a vector hbond_0 which has an angle jiH0 of 109.5  
                    '''
                    pkey_j = Htermed.bonded_nblist.getnbs(pkey_i)[0]
                    r_j =  Htermed.positions[pkey_j]
                    for pkey_jk in Htermed.bonded_nblist.getnbs(pkey_j):
                        if( pkey_jk != pkey_i ):
                            r_jk =  Htermed.positions[pkey_jk]
                    dr_CC = r_ij_array[0]
                    dr_CC_n = dr_CC/np.linalg.norm(dr_CC)
                    dr_CC_s = dr_CC_n*HCC_cos

                    dr_jk = Htermed.lat.deltasq_pos(r_j,r_jk)                    
                    cros_ik = np.cross(dr_CC,dr_jk)
                    cros_ik_n = cros_ik/np.linalg.norm(cros_ik)
                    cros_ik_s = cros_ik/np.linalg.norm(cros_ik)*HCC_sin

                    hbond_0 = cros_ik_s    + dr_CC_s
                    hpos_0 = r_i + hbond_0
                    
                    r_i0 = Htermed.lat.deltasq_pos(r_i,hpos_0)
                    
                    '''
                                 H1
                    theta=109.5 /   
                               /    
                        j----i ---- H0
                               \
                                \  
                                 \
                                  H2
                                  
                    so dr_CC_s is the same as H0 

                        H0(cros_ik_n)
                        | 
                        |    theta2 = 120.0
                        |
                        i  -------cros_jk
                       /  \
                      /    \
                     /      \
                    H2       H1 
                    
                    H1 is at  2pi/3 from cros_ik_n and cros_jk
                    H2 is at -2pi/3 from cros_ik_n and cros_jk

                    and cros_ijk_n is again scaled by sin(109.5)
                    
                    '''

                    cros_jk = np.cross(dr_CC,r_i0)
                    cros_jk_n = cros_jk/np.linalg.norm(cros_jk)
                    phi = 2.0*np.pi/3.0
                    cros_ijk = cros_jk_n*np.sin(phi) + cros_ik_n*np.cos(phi)
                    cros_ijk_n = cros_ijk/np.linalg.norm(cros_ijk)
                    cros_ijk_s = cros_ijk_n*HCC_sin
                    hbond_1 = cros_ijk_s + dr_CC_s

                    cros_ijk = -1.0*cros_jk_n*np.sin(phi) + cros_ik_n*np.cos(phi)
                    cros_ijk_n = cros_ijk/np.linalg.norm(cros_ijk)
                    cros_ijk_s = cros_ijk_n*HCC_sin
                    hbond_2 = cros_ijk_s + dr_CC_s

                    hpos_1 = r_i + hbond_1
                    hpos_2 = r_i + hbond_2

                    pt_H.properties["fftype"] = "HC"
                    Htermed.add_partpos(pt_H,hpos_0,deepcopy = True)
                    p_j = Htermed.n_particles -1 
                    Bond_iH = structure.Bond(pkey_i,p_j)
                    Htermed.add_bond(Bond_iH)
                    
                    pt_H.properties["fftype"] = "HC"
                    Htermed.add_partpos(pt_H,hpos_1,deepcopy = True)
                    p_j = Htermed.n_particles -1 
                    Bond_iH = structure.Bond(pkey_i,p_j)
                    Htermed.add_bond(Bond_iH)

                    pt_H.properties["fftype"] = "HC"
                    Htermed.add_partpos(pt_H,hpos_2,deepcopy = True)
                    p_j = Htermed.n_particles -1 
                    Bond_iH = structure.Bond(pkey_i,p_j)
                    Htermed.add_bond(Bond_iH)

                    #original_ref_mod.append(-1)
                    #pid_i_mod += 1 
                    #sub_ref_mod.append(pid_i_mod)

                    #  Add H-(i)-H angle
                    pid_H_j = Htermed.n_particles
                    a_i = structure.Angle( pid_H_j-1 ,pkey_i, pid_H_j )            
                    Htermed.add_angle(a_i)

                    #original_ref_mod.append(-1)
                    #pid_i_mod += 1 
                    #sub_ref_mod.append(pid_i_mod)
                                        
                else:
                    error_line =  " Nubmer of missing atoms %d has yet to be accounted for in groups.hterm \n"%(dB)
                    error_line +=  " {} -> {}".format(NNAB_o,NNAB_i)
                    Htermed.write_xyz("hterm_failed.xyz")
                    print error_line
                    #sys.exit(error_line)
                # Redo neighbor list
                Htermed.bonded_nblist.build_nblist(Htermed.particles,Htermed.bonds )
            
        #return Htermed #original_ref_mod,sub_ref_mod 


>hterm_group 3 1
 Nubmer of missing atoms 1 has yet to be accounted for in groups.hterm 
 3 -> 2
>hterm_group 3 2
>hterm_group 3 2


In [122]:
Htermed.write_xyz()

In [123]:
print Htermed.tag

residue_0_hterm


In [128]:
htermed = group_i.hterm_group()

>hterm_group 3 1
>hterm_group 3 2
>hterm_group 3 2


In [129]:
htermed.write_xyz()

In [130]:
htermed.tag

'residue_0_hterm'